<a href="https://colab.research.google.com/github/Shinchan9913/FlaskTut1/blob/main/Work_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Colab\ Notebooks/Kaggle/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download kazanova/sentiment140
!unzip sentiment140.zip

 89% 72.0M/80.9M [00:00<00:00, 131MB/s] 
100% 80.9M/80.9M [00:00<00:00, 108MB/s]
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', header=None, names=['target', 'id', 'date', 'flag', 'user', 'text'], encoding='latin-1')

df.head(5)

FileNotFoundError: ignored

In [3]:
vocab_size = 20000
embedding_dim = 32
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
df = df[['target', 'text']]

NameError: ignored

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
!unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Archive:  /root/nltk_data/corpora/wordnet.zip
   creating: /root/nltk_data/corpora/wordnet/
  inflating: /root/nltk_data/corpora/wordnet/lexnames  
  inflating: /root/nltk_data/corpora/wordnet/data.verb  
  inflating: /root/nltk_data/corpora/wordnet/index.adv  
  inflating: /root/nltk_data/corpora/wordnet/adv.exc  
  inflating: /root/nltk_data/corpora/wordnet/index.verb  
  inflating: /root/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /root/nltk_data/corpora/wordnet/data.adj  
  inflating: /root/nltk_data/corpora/wordnet/index.adj  
  inflating: /root/nltk_data/corpora/wordnet/LICENSE  
  inflating: /root/nltk_data/corpora/wordnet/citation.bib  
  inflating: /root/nltk_data/corpora/wordnet/noun.exc  
  inflating: /root/nltk_data/corpora/wordnet/verb.exc  
  inflating: /root/nltk_data/corpora/wordnet/README  
  inflating: /root/nltk_data/corpora/wordnet/index.sense  
  inflating: /root/nltk_data/corpora/wordnet/data.noun  
  inflating: /root/nltk_data/corpora/wordnet/data.adv  


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess(text):
    # Replace repeated punctuation signs with labels and add spaces
    text = re.sub(r'(\.{2,})', r' multistop ', text)
    text = re.sub(r'(\!{2,})', r' multiexclamation ', text)
    text = re.sub(r'(\?{2,})', r' multiquestion ', text)
    # Add spaces before and after single punctuation signs
    text = re.sub(r'(\.|\!|\?|\,)', r' ', text)
    
    # Lower case the text
    text = text.lower()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [w for w in words if not w in stop_words]
    text = ' '.join(words)
    
    # Lemmatize text
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words]
    text = ' '.join(words)

    return text

df["text"] = df["text"].apply(preprocess)

df.head()

,target,text
0,0,@switchfoot http://twitpic com/2y1zl - awww th...
1,0,upset can't update facebook texting multistop ...
2,0,@kenichan dived many time ball managed save 50...
3,0,whole body feel itchy like fire
4,0,@nationwideclass behaving i'm mad can't see


In [ ]:
def load_emoticons(emo_filename):
    # Load emoticons and their polarity from a file
    emoticon_dict = {}
    with open(emo_filename, 'r', encoding='latin-1') as file:
        for line in file:
            emoticon, polarity = line.strip().split('\t')
            emoticon_dict[emoticon] = polarity
    return emoticon_dict

# Load emoticons and their polarity from a file
emoticon_dict = load_emoticons('/content/EmoticonLookupTable.txt')

def replace_emoticons(text, emoticon_dict=emoticon_dict):
    # Replace emoticons with their polarity and delete neutral ones
    for emoticon, polarity in emoticon_dict.items():
        pattern = re.compile(re.escape(emoticon), re.IGNORECASE)
        if polarity == '1':
            text = pattern.sub("positive", text)
        elif polarity == '-1':
            text = pattern.sub("negative", text)
        else:
            text = pattern.sub('', text)
            
    text = re.sub(r'[^a-zA-Z\s]', '', text)
            
    return text.split()

df["text"] = df["text"].apply(replace_emoticons)
df.head()

,target,text
0,0,"[switchfoot, httpnegativetwitpic, comyzl, awww..."
1,0,"[upset, cant, update, facebook, texting, multi..."
2,0,"[kenichan, dived, many, time, ball, managed, s..."
3,0,"[whole, body, feel, itchy, like, fire]"
4,0,"[nationwideclass, behaving, im, mad, cant, see]"


In [ ]:
def load_slang(slang_filename):
    # Load emoticons and their polarity from a file
    slang_dict = {}
    with open(slang_filename, 'r', encoding='latin-1') as file:
        for line in file:
            slang, meaning = line.strip().split('\t')
            slang_dict[slang] = meaning
    return slang_dict

# Load emoticons and their polarity from a file
slang_dict = load_slang('/content/SlangLookupTable.txt')

def replace_slang(tokens, slang_dict=slang_dict):
    # Replace emoticons with their polarity and delete neutral ones
    for i, token in enumerate(tokens):
        if token in slang_dict:
            tokens[i] = slang_dict[token]
            
    return tokens

df["text"] = df["text"].apply(replace_slang)
df.head()

,target,text
0,0,"[switchfot, URL, comyzl, aw, thats, bummer, sh..."
1,0,"[upset, cant, update, facebok, texting, multis..."
2,0,"[kenichan, dived, many, time, ball, managed, s..."
3,0,"[whole, body, feel, itchy, like, fire]"
4,0,"[nationwideclas, behaving, im, mad, cant, see]"


In [ ]:
def label_user_topic(tokens):
    labeled_tokens = []
    for token in tokens:
        if token.startswith("@"):
            labeled_tokens.append("PERSON")
        elif token.startswith("#"):
            labeled_tokens.append("TOPIC")
        elif token.startswith("http"):
            labeled_tokens.append("URL")
        else:
            labeled_tokens.append(token)
    return labeled_tokens

df['text'] = df['text'].apply(label_user_topic)
df.head()

,target,text
0,0,"[switchfoot, URL, comyzl, awww, thats, bummer,..."
1,0,"[upset, cant, update, facebook, texting, multi..."
2,0,"[kenichan, dived, many, time, ball, managed, s..."
3,0,"[whole, body, feel, itchy, like, fire]"
4,0,"[nationwideclass, behaving, im, mad, cant, see]"


In [ ]:
from nltk.corpus import wordnet as wn

def reduce_word(word):
    # Check if the word is in Roget's Thesaurus
    synsets = wn.synsets(word)
    if synsets:
        return word
    
    # Iterate over the letters in the word, starting from the end
    for i in range(len(word)-1, 1, -1):
        # If the current letter is the same as the previous one,
        # remove the current letter and check if the resulting word
        # is in Roget's Thesaurus
        if word[i] == word[i-1]:
            word = word[:i] + word[i+1:]
            synsets = wn.synsets(word)
            if synsets:
                return "STRESSED " + word

        # If the current and previous letters are the same as the one before them,
        # remove the current letter and check if the resulting word
        # is in Roget's Thesaurus
        elif i > 2 and word[i] == word[i-2]:
            word = word[:i-1] + word[i:]
            synsets = wn.synsets(word)
            if synsets:
                return "STRESSED " + word
    
    # If no match is found, return the original word
    return word


def normalize_words(tokens):
    normalized_tokens = []
    for token in tokens:
        # Check if the token is a word
        if re.match(r'\b\w+\b', token):
            # Normalize the word
            normalized_word = reduce_word(token.lower())
            # If the normalized word is different from the original word,
            # add both versions to the list of tokens
            if normalized_word != token.lower():
                normalized_tokens.append(normalized_word)
            else:
                normalized_tokens.append(token)
        else:
            normalized_tokens.append(token)
            
#     normalized_tokens = [token.split() if 'STRESSED' in token else token for token in normalized_tokens]
#     normalized_tokens = [item if not isinstance(item, list) else item for sublist in normalized_tokens for item in sublist]
    return normalized_tokens

df["text"] = df["text"].apply(normalize_words)
df.tail()

,target,text
1599995,4,"[woke, school, best, feeling, ever]"
1599996,4,"[thewdb, com, cool, hear, old, walt, interview..."
1599997,4,"[ready, mojo, makeover, ask, detail]"
1599998,4,"[happy, th, birthday, boo, STRESSED all, time,..."
1599999,4,"[happy, charituesday, thenspc, sparkscharity, ..."


In [ ]:
!pip install twython
nltk.download('vader_lexicon')
nltk.download('opinion_lexicon')
nltk.download('sentiwordnet')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Unzipping corpora/opinion_lexicon.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import opinion_lexicon, sentiwordnet


def match_affect_words(tokens):
    sia = SentimentIntensityAnalyzer()
    positive_words = set(opinion_lexicon.positive())
    negative_words = set(opinion_lexicon.negative())
    hpositive_words = set(word for synset in sentiwordnet.senti_synsets('', 'a') for word, pos in synset.lemmas() 
                          if synset.pos_score() > 0.75 and word not in positive_words)
    hnegative_words = set(word for synset in sentiwordnet.senti_synsets('', 'a') for word, pos in synset.lemmas() 
                          if synset.neg_score() > 0.75 and word not in negative_words)
    affect_labels = {'positive': positive_words, 'negative': negative_words, 
                     'hpositive': hpositive_words, 'hnegative': hnegative_words}
    
    # create a mapping from affect words to labels
    affect_words = set(word for label_words in affect_labels.values() for word in label_words)
    word_to_label = {}
    for word in affect_words:
        scores = sia.polarity_scores(word)
        if scores['compound'] >= 0.5:
            word_to_label[word] = 'hpositive'
        elif scores['compound'] > -0.5 and scores['compound'] < 0.5:
            word_to_label[word] = 'positive' if word in positive_words else 'negative'
        else:
            word_to_label[word] = 'hnegative'
    
    # match tokens to affect labels
    affect_set = set(affect_words)
    matched_tokens = []
    for token in tokens:
        if token in affect_set:
            label = word_to_label[token]
            matched_tokens.append(label)
        else:
            matched_tokens.append(token)
    
    return matched_tokens



In [ ]:
df["text"] = df["text"].apply(match_affect_words)
df.tail()

KeyboardInterrupt: ignored

In [ ]:
# define the lists of negation, intensification and diminishment expressions
negation_list = ["no", "not", "never", "none", "nobody", "nowhere", "nothing", "neither", "nor", "cannot", "can't", "don't", "doesn't", "didn't", "won't", "wouldn't", "shouldn't", "couldn't", "isn't", "aren't", "ain't", "hate", "dislike", "disapprove", "disapprove of", "disagree", "disagree with", "reject", "rejects", "rejected", "refuse", "refuses", "refused", "never", "rarely", "seldom", "hardly", "scarcely", "barely"]
intensification_list = ["very", "extremely", "super", "really", "quite", "most", "more", "quite", "too", "enough", "so", "such", "just", "almost", "absolutely", "completely", "totally", "utterly", "highly", "deeply", "greatly", "seriously", "intensely", "especially", "exceedingly", "exceptionally", "particularly", "unusually", "incredibly", "undeniably", "undeniable", "emphatically", "decidedly", "really", "truly", "hugely", "mega", "ultra", "majorly", "extraordinarily", "mightily", "fully", "mightily", "perfectly", "thoroughly", "utterly", "all", "way", "significantly", "terribly", "awfully", "fantastically"]
diminishment_list = ["little", "slightly", "somewhat", "kind", "sort", "bit", "little", "moderately", "marginally", "fairly", "reasonably", "comparatively", "relatively", "tad", "touch", "extent"]


def match_modifier_words(tokens):
    matched_tokens = []
    for token in tokens:
        if token in negation_list:
            matched_tokens.append("negator")
        elif token in intensification_list:
            matched_tokens.append("intensifier")
        elif token in diminishment_list:
            matched_tokens.append("diminisher")
        else:
            matched_tokens.append(token)
    return matched_tokens

df["text"] = df["text"].apply(match_modifier_words)
df.tail()

,target,text
1599995,4,"[woke, school, best, feeling, ever]"
1599996,4,"[thewdb, com, cool, hear, old, walt, interview..."
1599997,4,"[ready, mojo, makeover, ask, detail]"
1599998,4,"[happy, th, birthday, boo, STRESSED all, time,..."
1599999,4,"[happy, charituesday, thenspc, sparkscharity, ..."


In [ ]:
data = df
data['text'] = df['text'].apply(lambda x: ' '.join(x))
df = data
df.head()

,target,text
0,0,switchfot URL comyzl aw thats bummer shoulda g...
1,0,upset cant update facebok texting multistop mi...
2,0,kenichan dived many time ball managed save res...
3,0,whole body feel itchy like fire
4,0,nationwideclas behaving im mad cant see


In [ ]:
X = df["text"]
y = df["target"]

y = y.replace(4,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [4]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)


In [ ]:
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compiling the module
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
num_epochs = 1
history = model.fit(X_train_padded, y_train, epochs=num_epochs,batch_size=32, validation_data=(X_test_padded, y_test))

40000/40000 [==============================] - 800s 20ms/step - loss: 0.4574 - accuracy: 0.7847 - val_loss: 0.4351 - val_accuracy: 0.7956


In [ ]:
test_loss, test_acc = model.evaluate(X_test_padded, y_test, verbose=2)
print("Test Accuracy: ", test_acc)

10000/10000 - 67s - loss: 0.4351 - accuracy: 0.7956 - 67s/epoch - 7ms/step
Test Accuracy:  0.795606255531311


In [ ]:
# Test with input entry
entry = ["I'm not sure if I don't like it or if I just don't understand it."]
test_text = tokenizer.texts_to_sequences(entry)
test_text_padded = pad_sequences(test_text, maxlen=max_length, padding=padding_type, truncating=trunc_type)

prediction = model.predict(test_text_padded)

#0 : bad
#1 : good
print(prediction)

# entry = ['I hate this music so bad ! I just want to sleep rn']
# test_text = tokenizer.texts_to_sequences(entry)
# test_text_padded = pad_sequences(test_text, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# prediction = model.predict(test_text_padded)


# #0 : bad
# #1 : good
# print(prediction)

1/1 [==============================] - 1s 1s/step
[[0.468318]]


In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/work_1.keras')

In [5]:
load_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/work_1.keras')

# Show the model architecture
load_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           640000    
                                                                 
 bidirectional (Bidirectiona  (None, 100, 128)         49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [17]:
# Test with input entry
entry = ["I am very angry with her"]
tokenizer.fit_on_texts(entry)
test_text = tokenizer.texts_to_sequences(entry)
test_text_padded = pad_sequences(test_text, maxlen=max_length, padding=padding_type, truncating='post')

prediction = load_model.predict(test_text_padded)

#0 : bad
#1 : good
print(prediction)

1/1 [==============================] - 0s 26ms/step
[[0.8706128]]
